In [ ]:
def simulate_lifetime_earnings(grossSalaryPA, 
                               N, 
                               principal, 
                               employment_start, 
                               method="monte carlo", 
                               increases=None):
    '''
    Simulates payment of student debt across career based 
    on monthly tax payments according to estimated salary 
    and simultaneous compounding interest of loan
    '''
    
    salary = [] # store salary trajectory for plotting
    payments_towards = 0 # cumulative total of payments made towards loan across career
    salary.append(grossSalaryPA)
    
    ## Simulation start and end points
    today = pd.Timestamp(employment_start)
    start = today
    edate = today + timedelta(days=N*365) # N years of payments
    delta = timedelta(days=1)
    offset = BMonthEnd() # date each month that tax is paid
    
    cumulativeTotal = principal
    cumulativeList = [] # store the value of the loan at each timestep
    cumulativeList.append(cumulativeTotal)
    
    ## Simulate through N years of payments
    ## interest compounds each day
    ## payment made on last working day of the month
    while today <= edate:
        if method == "monte carlo" and increases is not None:
            if ((today - start).days/365)%5 == 0 and (today - start).days > 0:
                grossSalaryPA *= increases[(today - start).days/365/5]
                
        elif method == "monte carlo" and increases is None:
            print("Please supply monte carlo simulated salary increases")
            
        if method == "median":
            if (today - start).days%365 == 0 and (today - start).days/365 in male_HE.index:
                grossSalaryPA = male_HE.loc[(today - start).days/365]['Earning']

        cumulativeTotal *= (1+(0.05/365)) # apply interest on previous payments before new payment
        
        if today == offset.rollforward(today) and today.month == offset.rollforward(today).month: # if today in sim is last working day
            cumulativeTotal -= calc_student_finance_PM(grossSalaryPA) # payment made to student finance based on gross salary
            payments_towards += calc_student_finance_PM(grossSalaryPA) # this payment is added to cumulative sum
            cumulativeList.append(cumulativeTotal)
            salary.append(grossSalaryPA)
            
        today += delta

    return cumulativeList, salary, payments_towards